<a href="https://colab.research.google.com/github/JoeRatterman/FantasyLineup/blob/master/FantasyLineup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pulp
!pip install pyspark

    100% |████████████████████████████████| 13.6MB 2.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/3b/f1/f5/f6d0aac56dd154c2593d3fa5e901aa9bc32a51bd8594b4bf8e
Successfully built pulp


In [0]:
from pulp import *
import numpy as np
import itertools
import pandas as pd
import os
from google.colab import files
import io


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name='Users\joera\Documents\Grad Case Studies\PuLP Practice',
      length=len(uploaded[fn])))

Saving PlayersData.csv to PlayersData.csv
User uploaded file "Users\joera\Documents\Grad Case Studies\PuLP Practice" with length 2925 bytes


In [0]:
players = pd.read_csv(io.BytesIO(uploaded['PlayersData.csv']))
print(players.head())

   Number                Name Position  Salary  ProjectedScore
0       1           Tom Brady       QB    8369              12
1       2          Cam Newton       QB    7826              13
2       3         Andrew Luck       QB   12597              21
3       4  Patrick Mahomes II       QB    9522              10
4       5         Andy Dalton       QB   12747               4


In [0]:
prob = pulp.LpProblem("Fantasy", pulp.LpMaximize)
P = range(100)
x = pulp.LpVariable.matrix("x", list(P) ,lowBound = 0, upBound = 1,cat = pulp.LpInteger)

In [0]:
score = list(players['ProjectedScore'])
cost = list(players['Salary'])

In [0]:
#obj. function
prob += sum(score[p]*x[p] for p in P)

In [0]:
#Filter / Sets
QBf = players['Position'] == 'QB'
QB = P[0:26]

RBf = players['Position'] == 'RB'
RB = P[25:56]

WRf = players['Position'] == 'WR'
WR = P[63:97]

TEf = players['Position'] == 'TE'
TE = P[56:63]

DSTf = players['Position'] == 'DST'
DST = P[97:]

In [0]:
# Constraints
prob += sum(x[p] for p in P) == 6 
prob += sum(x[p] for p in QB) == 1
prob += sum(x[p]*cost[p] for p in P) <=50000
prob += sum(x[p] for p in RB) >= 1
prob += sum(x[p] for p in TE) >= 1
prob += sum(x[p] for p in WR) >= 1
prob += sum(x[p] for p in RB) <= 2
prob += sum(x[p] for p in TE) <= 2
prob += sum(x[p] for p in WR) <= 2
prob += sum(x[p] for p in DST) == 1

In [50]:
prob.solve()
print("Status:", pulp.LpStatus[prob.status])
lineup = [players['Name'][p] for p in P if x[p].varValue]
projected = sum([players['ProjectedScore'][p] for p in P if x[p].varValue])
salary = sum([players['Salary'][p] for p in P if x[p].varValue])
print("The lineup is: " + str(lineup))
print("The total projected score is: " + str(projected))
print("The total salary is: " + str(salary))


Status: Optimal
The lineup is: ['Jimmy Garoppolo', 'Alvin Kamara', 'Kenyan Drake', 'Evan Engram', 'Sterling Shepard', 'Vikings ']
The total projected score is: 143
The total salary is: 48631
